In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
print ("PACKAGES LOADED")

PACKAGES LOADED


In [3]:
mnist      = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST loaded")


Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


In [27]:
x = tf.placeholder("float", [None, n_input]) 
y = tf.placeholder("float", [None, n_classes])  # None is for infinite 

n_hidden_1 = 256
n_hidden_2 = 128
n_hidden_3 = 64
n_input = 784
n_classes = 10
stddev = 0.1

weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'h3' : tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])), #stddev default : 1
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'b3' : tf.Variable(tf.random_normal([n_hidden_3])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perception(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']) , _biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']) , _biases['b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['h3']), _biases['b3']))
    return tf.matmul(layer_3, _weights['out']) + _biases['out']

pred = multilayer_perception(x, weights, biases)


In [28]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=pred))
optm = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr= tf.reduce_mean(tf.cast(corr,"float"))

In [29]:
init = tf.global_variables_initializer()


In [30]:
training_epochs = 50
batch_size      = 100
display_step    = 5
# SESSION
sess = tf.Session()
sess.run(init)
# MINI-BATCH LEARNING
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch): 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        feeds = {x: batch_xs, y: batch_ys}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    # DISPLAY
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/050 cost: 0.983101853 train_acc: 0.930 test_acc: 0.917
Epoch: 005/050 cost: 0.067318746 train_acc: 0.990 test_acc: 0.972
Epoch: 010/050 cost: 0.020621170 train_acc: 1.000 test_acc: 0.977
Epoch: 015/050 cost: 0.006824123 train_acc: 1.000 test_acc: 0.976
Epoch: 020/050 cost: 0.002779052 train_acc: 1.000 test_acc: 0.975
Epoch: 025/050 cost: 0.000984152 train_acc: 1.000 test_acc: 0.977
Epoch: 030/050 cost: 0.000256641 train_acc: 1.000 test_acc: 0.980
Epoch: 035/050 cost: 0.002759294 train_acc: 1.000 test_acc: 0.979
Epoch: 040/050 cost: 0.000081564 train_acc: 1.000 test_acc: 0.981
Epoch: 045/050 cost: 0.000034985 train_acc: 1.000 test_acc: 0.981
DONE
